In [25]:
#Cargamos librerias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.special as special
import scipy.optimize as curve_fit
import seaborn as sns
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [26]:
#Cargar archivo csv
df = pd.read_csv('50_sin_nulos_ni_atipicos_MexicoCity_DistritoFederal_Mexico.csv')
df = df.drop(['Unnamed: 0'], axis = 1)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26582 entries, 0 to 26581
Data columns (total 50 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   host_name                    26582 non-null  object 
 1   host_since                   26582 non-null  object 
 2   host_location                26582 non-null  object 
 3   host_response_time           26582 non-null  object 
 4   host_response_rate           26582 non-null  object 
 5   host_is_superhost            26582 non-null  object 
 6   host_neighbourhood           26582 non-null  object 
 7   host_verifications           26582 non-null  object 
 8   host_has_profile_pic         26582 non-null  object 
 9   host_identity_verified       26582 non-null  object 
 10  neighbourhood                26582 non-null  object 
 11  neighbourhood_cleansed       26582 non-null  object 
 12  property_type                26582 non-null  object 
 13  room_type       

**Remplazar valores**

In [27]:
#Verificar los valores sin repertirse de una columna
unico1 = np.unique(df['host_is_superhost'])
unico2 = np.unique(df['has_availability'])
unico3 = np.unique(df['instant_bookable'])
unico4 = np.unique(df['property_type'])
unico5 = np.unique(df['room_type'])
unico6 = np.unique(df['host_identity_verified'])
unico7 = np.unique(df['host_location'])
unico8 = np.unique(df['host_response_time'])
unico9 = np.unique(df['bedrooms'])
unico10 = np.unique(df['host_has_profile_pic'])
print(unico1)
print(unico2)
print(unico3)
print(unico4)
print(unico5)
print(unico6)
print(unico7)
print(unico8)
print(unico9)
print(unico10)

['Unknown' 'f' 't']
['Unknown' 'f' 't']
['f' 't']
['Boat' 'Campsite' 'Casa particular' 'Castle' 'Dome' 'Earthen home'
 'Entire bungalow' 'Entire cabin' 'Entire chalet' 'Entire condo'
 'Entire cottage' 'Entire guest suite' 'Entire guesthouse' 'Entire home'
 'Entire home/apt' 'Entire hostel' 'Entire in-law' 'Entire loft'
 'Entire place' 'Entire rental unit' 'Entire serviced apartment'
 'Entire townhouse' 'Entire vacation home' 'Entire villa' 'Farm stay'
 'Holiday park' 'Hut' 'Private room' 'Private room in barn'
 'Private room in bed and breakfast' 'Private room in bungalow'
 'Private room in cabin' 'Private room in casa particular'
 'Private room in castle' 'Private room in chalet' 'Private room in condo'
 'Private room in cottage' 'Private room in dome' 'Private room in dorm'
 'Private room in earthen home' 'Private room in farm stay'
 'Private room in floor' 'Private room in guest suite'
 'Private room in guesthouse' 'Private room in home'
 'Private room in hostel' 'Private room in ho

In [28]:
#Convierto  una variable a dicotomica
df['host_is_superhost'] =df['host_is_superhost'].replace(["Unknown"], "f")
df['has_availability'] =df['has_availability'].replace(["Unknown"], "f")
#df['instant_bookable'] =df['instant_bookable'].replace(["Unknown"], "f")
df['property_type'] = df['property_type'].mask(df['property_type'] != "Entire rental unit", "Other property")
df['room_type'] =df['room_type'].replace(["Hotel room", "Private room", "Shared room"], "Other room")
#df['host_identity_verified'] =df['host_identity_verified'].replace(["Unknown"], "f")
df['host_location'] =df['host_location'].mask(df['host_location'] != "Ciudad de Mexico, Mexico", "Other location")
df['host_response_time'] =df['host_response_time'].replace(["Not defined", "a few days or more", "within a day", "within a few hours"], "More than one hour")
df['bedrooms'] =df['bedrooms'].replace([1, 1.5, 3, 1.4], 0)
#df['host_has_profile_pic'] =df['host_has_profile_pic'].replace(["Unknown"], "f")

In [29]:
#Corroboramos valores nulos
valores_nulos = df.isnull().sum().sum()
valores_nulos

0

**Prediccion host_is_superhost**

In [30]:
#Declaramos las variables dependientes e idependientes para la regresion logistica
Vars_Indep = df[['review_scores_checkin', 'review_scores_cleanliness', 'review_scores_communication']] #Numericas
Var_Dep = df['host_is_superhost'] #Dicotomica
#Redefinimos las variables
X = Vars_Indep
y = Var_Dep
#Dividimos el conjunto de datos en la parte de entrenamiento y prueba:
X_train, X_test, y_train, y_test =train_test_split(X, y, test_size= 0.3, random_state=None)
#Se escalan todos los datos
escalar = StandardScaler()
#Para realizar el escalamiento de las variables "X" tanto de entrenamiento como de prueba, utilizaremos fit_transform
X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)
#Definimos el algoritmo a utilizar
from sklearn.linear_model import LogisticRegression
algoritmo = LogisticRegression()
#Entrenamos al modelo
algoritmo.fit(X_train, y_train)
#Realizamos una prediccion
y_pred = algoritmo.predict(X_test)
#Verifico la matriz de confusion
from sklearn.metrics import confusion_matrix
matriz = confusion_matrix(y_test, y_pred)
print('Matriz de Confusion: ')
print(matriz)
#Calculo la precision del modelo
from sklearn.metrics import precision_score
precision = precision_score(y_test, y_pred, average="binary", pos_label="t")
print('Precision del modelo: ', precision)
#Calculamos la exactitud del modelo
from sklearn.metrics import accuracy_score
exactitud = accuracy_score(y_test, y_pred)
print('Exactitud del modelo: ', exactitud)
#Calculo la sensibilidad del modelo
from sklearn.metrics import recall_score
sensibilidad = recall_score(y_test, y_pred, average="binary", pos_label="t")
print("Sensibilidad del modelo: ", sensibilidad)

Matriz de Confusion: 
[[5069    0]
 [2906    0]]
Precision del modelo:  0.0
Exactitud del modelo:  0.6356112852664577
Sensibilidad del modelo:  0.0


c:\Users\lmanu\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


**Prediccion has_availability**

In [31]:
#Declaramos las variables dependientes e idependientes para la regresion logistica
Vars_Indep = df[['availability_30', 'availability_60', 'availability_90']] #Numericas
Var_Dep = df['has_availability'] #Dicotomica
#Redefinimos las variables
X = Vars_Indep
y = Var_Dep
#Dividimos el conjunto de datos en la parte de entrenamiento y prueba:
X_train, X_test, y_train, y_test =train_test_split(X, y, test_size= 0.3, random_state=None)
#Se escalan todos los datos
escalar = StandardScaler()
#Para realizar el escalamiento de las variables "X" tanto de entrenamiento como de prueba, utilizaremos fit_transform
X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)
#Definimos el algoritmo a utilizar
from sklearn.linear_model import LogisticRegression
algoritmo = LogisticRegression()
#Entrenamos al modelo
algoritmo.fit(X_train, y_train)
#Realizamos una prediccion
y_pred = algoritmo.predict(X_test)
#Verifico la matriz de confusion
from sklearn.metrics import confusion_matrix
matriz = confusion_matrix(y_test, y_pred)
print('Matriz de Confusion: ')
print(matriz)
#Calculo la precision del modelo
from sklearn.metrics import precision_score
precision = precision_score(y_test, y_pred, average="binary", pos_label="t")
print('Precision del modelo: ', precision)
#Calculamos la exactitud del modelo
from sklearn.metrics import accuracy_score
exactitud = accuracy_score(y_test, y_pred)
print('Exactitud del modelo: ', exactitud)
#Calculo la sensibilidad del modelo
from sklearn.metrics import recall_score
sensibilidad = recall_score(y_test, y_pred, average="binary", pos_label="t")
print("Sensibilidad del modelo: ", sensibilidad)

Matriz de Confusion: 
[[   0  324]
 [   0 7651]]
Precision del modelo:  0.9593730407523511
Exactitud del modelo:  0.9593730407523511
Sensibilidad del modelo:  1.0


**Prediccion instant_bookable**

In [32]:
#Declaramos las variables dependientes e idependientes para la regresion logistica
Vars_Indep = df[['host_acceptance_rate', 'price', 'availability_365']] #Numericas
Var_Dep = df['instant_bookable'] #Dicotomica
#Redefinimos las variables
X = Vars_Indep
y = Var_Dep
#Dividimos el conjunto de datos en la parte de entrenamiento y prueba:
X_train, X_test, y_train, y_test =train_test_split(X, y, test_size= 0.3, random_state=None)
#Se escalan todos los datos
escalar = StandardScaler()
#Para realizar el escalamiento de las variables "X" tanto de entrenamiento como de prueba, utilizaremos fit_transform
X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)
#Definimos el algoritmo a utilizar
from sklearn.linear_model import LogisticRegression
algoritmo = LogisticRegression()
#Entrenamos al modelo
algoritmo.fit(X_train, y_train)
#Realizamos una prediccion
y_pred = algoritmo.predict(X_test)
#Verifico la matriz de confusion
from sklearn.metrics import confusion_matrix
matriz = confusion_matrix(y_test, y_pred)
print('Matriz de Confusion: ')
print(matriz)
#Calculo la precision del modelo
from sklearn.metrics import precision_score
precision = precision_score(y_test, y_pred, average="binary", pos_label="t")
print('Precision del modelo: ', precision)
#Calculamos la exactitud del modelo
from sklearn.metrics import accuracy_score
exactitud = accuracy_score(y_test, y_pred)
print('Exactitud del modelo: ', exactitud)
#Calculo la sensibilidad del modelo
from sklearn.metrics import recall_score
sensibilidad = recall_score(y_test, y_pred, average="binary", pos_label="t")
print("Sensibilidad del modelo: ", sensibilidad)

Matriz de Confusion: 
[[3844 1004]
 [1473 1654]]
Precision del modelo:  0.6222723852520692
Exactitud del modelo:  0.6894043887147335
Sensibilidad del modelo:  0.5289414774544292


**Prediccion property_type**

In [33]:
#Declaramos las variables dependientes e idependientes para la regresion logistica
Vars_Indep = df[['review_scores_location', 'accommodates', 'price']] #Numericas
Var_Dep = df['property_type'] #Dicotomica
#Redefinimos las variables
X = Vars_Indep
y = Var_Dep
#Dividimos el conjunto de datos en la parte de entrenamiento y prueba:
X_train, X_test, y_train, y_test =train_test_split(X, y, test_size= 0.3, random_state=None)
#Se escalan todos los datos
escalar = StandardScaler()
#Para realizar el escalamiento de las variables "X" tanto de entrenamiento como de prueba, utilizaremos fit_transform
X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)
#Definimos el algoritmo a utilizar
from sklearn.linear_model import LogisticRegression
algoritmo = LogisticRegression()
#Entrenamos al modelo
algoritmo.fit(X_train, y_train)
#Realizamos una prediccion
y_pred = algoritmo.predict(X_test)
#Verifico la matriz de confusion
from sklearn.metrics import confusion_matrix
matriz = confusion_matrix(y_test, y_pred)
print('Matriz de Confusion: ')
print(matriz)
#Calculo la precision del modelo
from sklearn.metrics import precision_score
precision = precision_score(y_test, y_pred, average="binary", pos_label="Entire rental unit")
print('Precision del modelo: ', precision)
#Calculamos la exactitud del modelo
from sklearn.metrics import accuracy_score
exactitud = accuracy_score(y_test, y_pred)
print('Exactitud del modelo: ', exactitud)
#Calculo la sensibilidad del modelo
from sklearn.metrics import recall_score
sensibilidad = recall_score(y_test, y_pred, average="binary", pos_label="Entire rental unit")
print("Sensibilidad del modelo: ", sensibilidad)

Matriz de Confusion: 
[[1037 2003]
 [ 835 4100]]
Precision del modelo:  0.5539529914529915
Exactitud del modelo:  0.6441379310344828
Sensibilidad del modelo:  0.34111842105263157


**Prediccion room_type**

In [34]:
#Declaramos las variables dependientes e idependientes para la regresion logistica
Vars_Indep = df[['price', 'accommodates', 'bathrooms']] #Numericas
Var_Dep = df['room_type'] #Dicotomica
#Redefinimos las variables
X = Vars_Indep
y = Var_Dep
#Dividimos el conjunto de datos en la parte de entrenamiento y prueba:
X_train, X_test, y_train, y_test =train_test_split(X, y, test_size= 0.3, random_state=None)
#Se escalan todos los datos
escalar = StandardScaler()
#Para realizar el escalamiento de las variables "X" tanto de entrenamiento como de prueba, utilizaremos fit_transform
X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)
#Definimos el algoritmo a utilizar
from sklearn.linear_model import LogisticRegression
algoritmo = LogisticRegression()
#Entrenamos al modelo
algoritmo.fit(X_train, y_train)
#Realizamos una prediccion
y_pred = algoritmo.predict(X_test)
#Verifico la matriz de confusion
from sklearn.metrics import confusion_matrix
matriz = confusion_matrix(y_test, y_pred)
print('Matriz de Confusion: ')
print(matriz)
#Calculo la precision del modelo
from sklearn.metrics import precision_score
precision = precision_score(y_test, y_pred, average="binary", pos_label="Entire home/apt")
print('Precision del modelo: ', precision)
#Calculamos la exactitud del modelo
from sklearn.metrics import accuracy_score
exactitud = accuracy_score(y_test, y_pred)
print('Exactitud del modelo: ', exactitud)
#Calculo la sensibilidad del modelo
from sklearn.metrics import recall_score
sensibilidad = recall_score(y_test, y_pred, average="binary", pos_label="Entire home/apt")
print("Sensibilidad del modelo: ", sensibilidad)

Matriz de Confusion: 
[[4514  686]
 [ 874 1901]]
Precision del modelo:  0.8377876763177431
Exactitud del modelo:  0.8043887147335423
Sensibilidad del modelo:  0.8680769230769231


**Prediccion host_identity_verified**

In [35]:
#Declaramos las variables dependientes e idependientes para la regresion logistica
Vars_Indep = df[['review_scores_value', 'host_total_listings_count', 'number_of_reviews']] #Numericas
Var_Dep = df['host_identity_verified'] #Dicotomica
#Redefinimos las variables
X = Vars_Indep
y = Var_Dep
#Dividimos el conjunto de datos en la parte de entrenamiento y prueba:
X_train, X_test, y_train, y_test =train_test_split(X, y, test_size= 0.3, random_state=None)
#Se escalan todos los datos
escalar = StandardScaler()
#Para realizar el escalamiento de las variables "X" tanto de entrenamiento como de prueba, utilizaremos fit_transform
X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)
#Definimos el algoritmo a utilizar
from sklearn.linear_model import LogisticRegression
algoritmo = LogisticRegression()
#Entrenamos al modelo
algoritmo.fit(X_train, y_train)
#Realizamos una prediccion
y_pred = algoritmo.predict(X_test)
#Verifico la matriz de confusion
from sklearn.metrics import confusion_matrix
matriz = confusion_matrix(y_test, y_pred)
print('Matriz de Confusion: ')
print(matriz)
#Calculo la precision del modelo
from sklearn.metrics import precision_score
precision = precision_score(y_test, y_pred, average="binary", pos_label="t")
print('Precision del modelo: ', precision)
#Calculamos la exactitud del modelo
from sklearn.metrics import accuracy_score
exactitud = accuracy_score(y_test, y_pred)
print('Exactitud del modelo: ', exactitud)
#Calculo la sensibilidad del modelo
from sklearn.metrics import recall_score
sensibilidad = recall_score(y_test, y_pred, average="binary", pos_label="t")
print("Sensibilidad del modelo: ", sensibilidad)

Matriz de Confusion: 
[[   0  348]
 [   0 7627]]
Precision del modelo:  0.9563636363636364
Exactitud del modelo:  0.9563636363636364
Sensibilidad del modelo:  1.0


**Prediccion host_location**

In [36]:
#Declaramos las variables dependientes e idependientes para la regresion logistica
Vars_Indep = df[['review_scores_communication', 'review_scores_location', 'review_scores_cleanliness']] #Numericas
Var_Dep = df['host_location'] #Dicotomica
#Redefinimos las variables
X = Vars_Indep
y = Var_Dep
#Dividimos el conjunto de datos en la parte de entrenamiento y prueba:
X_train, X_test, y_train, y_test =train_test_split(X, y, test_size= 0.3, random_state=None)
#Se escalan todos los datos
escalar = StandardScaler()
#Para realizar el escalamiento de las variables "X" tanto de entrenamiento como de prueba, utilizaremos fit_transform
X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)
#Definimos el algoritmo a utilizar
from sklearn.linear_model import LogisticRegression
algoritmo = LogisticRegression()
#Entrenamos al modelo
algoritmo.fit(X_train, y_train)
#Realizamos una prediccion
y_pred = algoritmo.predict(X_test)
#Verifico la matriz de confusion
from sklearn.metrics import confusion_matrix
matriz = confusion_matrix(y_test, y_pred)
print('Matriz de Confusion: ')
print(matriz)
#Calculo la precision del modelo
from sklearn.metrics import precision_score
precision = precision_score(y_test, y_pred, average="binary", pos_label="Ciudad de Mexico, Mexico")
print('Precision del modelo: ', precision)
#Calculamos la exactitud del modelo
from sklearn.metrics import accuracy_score
exactitud = accuracy_score(y_test, y_pred)
print('Exactitud del modelo: ', exactitud)
#Calculo la sensibilidad del modelo
from sklearn.metrics import recall_score
sensibilidad = recall_score(y_test, y_pred, average="binary", pos_label="Ciudad de Mexico, Mexico")
print("Sensibilidad del modelo: ", sensibilidad)

Matriz de Confusion: 
[[   0   64]
 [   0 7911]]
Precision del modelo:  0.0
Exactitud del modelo:  0.991974921630094
Sensibilidad del modelo:  0.0


c:\Users\lmanu\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


**Prediccion host_response_time**

In [37]:
#Declaramos las variables dependientes e idependientes para la regresion logistica
Vars_Indep = df[['review_scores_communication', 'review_scores_value', 'price']] #Numericas
Var_Dep = df['host_response_time'] #Dicotomica
#Redefinimos las variables
X = Vars_Indep
y = Var_Dep
#Dividimos el conjunto de datos en la parte de entrenamiento y prueba:
X_train, X_test, y_train, y_test =train_test_split(X, y, test_size= 0.3, random_state=None)
#Se escalan todos los datos
escalar = StandardScaler()
#Para realizar el escalamiento de las variables "X" tanto de entrenamiento como de prueba, utilizaremos fit_transform
X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)
#Definimos el algoritmo a utilizar
from sklearn.linear_model import LogisticRegression
algoritmo = LogisticRegression()
#Entrenamos al modelo
algoritmo.fit(X_train, y_train)
#Realizamos una prediccion
y_pred = algoritmo.predict(X_test)
#Verifico la matriz de confusion
from sklearn.metrics import confusion_matrix
matriz = confusion_matrix(y_test, y_pred)
print('Matriz de Confusion: ')
print(matriz)
#Calculo la precision del modelo
from sklearn.metrics import precision_score
precision = precision_score(y_test, y_pred, average="binary", pos_label="within an hour")
print('Precision del modelo: ', precision)
#Calculamos la exactitud del modelo
from sklearn.metrics import accuracy_score
exactitud = accuracy_score(y_test, y_pred)
print('Exactitud del modelo: ', exactitud)
#Calculo la sensibilidad del modelo
from sklearn.metrics import recall_score
sensibilidad = recall_score(y_test, y_pred, average="binary", pos_label="within an hour")
print("Sensibilidad del modelo: ", sensibilidad)

Matriz de Confusion: 
[[   0 2310]
 [   0 5665]]
Precision del modelo:  0.7103448275862069
Exactitud del modelo:  0.7103448275862069
Sensibilidad del modelo:  1.0


**Prediccion bedrooms**

In [38]:
#Declaramos las variables dependientes e idependientes para la regresion logistica
Vars_Indep = df[['accommodates', 'bathrooms', 'price']] #Numericas
Var_Dep = df['bedrooms'] #Dicotomica
#Redefinimos las variables
X = Vars_Indep
y = Var_Dep
#Dividimos el conjunto de datos en la parte de entrenamiento y prueba:
X_train, X_test, y_train, y_test =train_test_split(X, y, test_size= 0.3, random_state=None)
#Se escalan todos los datos
escalar = StandardScaler()
#Para realizar el escalamiento de las variables "X" tanto de entrenamiento como de prueba, utilizaremos fit_transform
X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)
#Definimos el algoritmo a utilizar
from sklearn.linear_model import LogisticRegression
algoritmo = LogisticRegression()
#Entrenamos al modelo
algoritmo.fit(X_train, y_train)
#Realizamos una prediccion
y_pred = algoritmo.predict(X_test)
#Verifico la matriz de confusion
from sklearn.metrics import confusion_matrix
matriz = confusion_matrix(y_test, y_pred)
print('Matriz de Confusion: ')
print(matriz)
#Calculo la precision del modelo
from sklearn.metrics import precision_score
precision = precision_score(y_test, y_pred, average="binary", pos_label=2)
print('Precision del modelo: ', precision)
#Calculamos la exactitud del modelo
from sklearn.metrics import accuracy_score
exactitud = accuracy_score(y_test, y_pred)
print('Exactitud del modelo: ', exactitud)
#Calculo la sensibilidad del modelo
from sklearn.metrics import recall_score
sensibilidad = recall_score(y_test, y_pred, average="binary", pos_label=2)
print("Sensibilidad del modelo: ", sensibilidad)

Matriz de Confusion: 
[[5438  459]
 [1369  709]]
Precision del modelo:  0.6070205479452054
Exactitud del modelo:  0.7707836990595611
Sensibilidad del modelo:  0.3411934552454283


**Prediccion host_has_profile_pic**

In [39]:
#Declaramos las variables dependientes e idependientes para la regresion logistica
Vars_Indep = df[['review_scores_communication', 'review_scores_value', 'reviews_per_month']] #Numericas
Var_Dep = df['host_has_profile_pic'] #Dicotomica
#Redefinimos las variables
X = Vars_Indep
y = Var_Dep
#Dividimos el conjunto de datos en la parte de entrenamiento y prueba:
X_train, X_test, y_train, y_test =train_test_split(X, y, test_size= 0.3, random_state=None)
#Se escalan todos los datos
escalar = StandardScaler()
#Para realizar el escalamiento de las variables "X" tanto de entrenamiento como de prueba, utilizaremos fit_transform
X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)
#Definimos el algoritmo a utilizar
from sklearn.linear_model import LogisticRegression
algoritmo = LogisticRegression()
#Entrenamos al modelo
algoritmo.fit(X_train, y_train)
#Realizamos una prediccion
y_pred = algoritmo.predict(X_test)
#Verifico la matriz de confusion
from sklearn.metrics import confusion_matrix
matriz = confusion_matrix(y_test, y_pred)
print('Matriz de Confusion: ')
print(matriz)
#Calculo la precision del modelo
from sklearn.metrics import precision_score
precision = precision_score(y_test, y_pred, average="binary", pos_label="t")
print('Precision del modelo: ', precision)
#Calculamos la exactitud del modelo
from sklearn.metrics import accuracy_score
exactitud = accuracy_score(y_test, y_pred)
print('Exactitud del modelo: ', exactitud)
#Calculo la sensibilidad del modelo
from sklearn.metrics import recall_score
sensibilidad = recall_score(y_test, y_pred, average="binary", pos_label="t")
print("Sensibilidad del modelo: ", sensibilidad)

Matriz de Confusion: 
[[   0  145]
 [   0 7830]]
Precision del modelo:  0.9818181818181818
Exactitud del modelo:  0.9818181818181818
Sensibilidad del modelo:  1.0
